In [1]:
import numpy as np
import pandas as pd

In [5]:
train = pd.read_csv('../data/amz-book/train (2).csv')
test = pd.read_csv('/kaggle/input/amz-book/test (3).csv')

In [6]:
train.head()

,score,time,id,user_id
0,5.0,1355702400,92408,41587
1,5.0,1356220800,32520,24303
2,4.0,1330560000,112766,56189
3,4.0,1006128000,97297,3294
4,5.0,1290816000,20794,50340


In [7]:
!git clone https://github.com/microsoft/recommenders.git ./recommenders_microsoft

Cloning into './recommenders_microsoft'...
remote: Enumerating objects: 42928, done.
remote: Counting objects: 100% (6452/6452), done.
remote: Compressing objects: 100% (2181/2181), done.
remote: Total 42928 (delta 4259), reused 5748 (delta 3877), pack-reused 36476 (from 1)
Receiving objects: 100% (42928/42928), 218.13 MiB | 34.00 MiB/s, done.
Resolving deltas: 100% (28904/28904), done.


In [8]:
!pip install retrying

In [9]:
import sys
import os
sys.path.insert(0, './recommenders_microsoft')
sys.path.insert(0, './recommenders_microsoft/recommenders')

In [10]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
Pandas version: 2.2.2
Tensorflow version: 2.16.1


In [11]:
train = train.rename(columns = {'user_id':'userID', 'id':'itemID','score':'rating'})
test = test.rename(columns = {'user_id':'userID', 'id':'itemID','score':'rating'})

In [12]:
data = ImplicitCF(
    train=train, test=test, seed=0,
    col_user='userID',
    col_item='itemID',
    col_rating='rating'
)

In [16]:
yaml_file = './recommenders_microsoft/examples/07_tutorials/KDD2020-tutorial/lightgcn.yaml'


hparams = prepare_hparams(yaml_file,                          
                          learning_rate=0.01,
                          eval_epoch=1,
                          top_k=10,
                          save_model=True,
                          epochs=50,
                          save_epoch=1
                         )

In [17]:
model = LightGCN(hparams, data, seed=0)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [18]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Save model to path /kaggle/working/tests/resources/deeprec/lightgcn/model/lightgcn_model/epoch_1
Epoch 1 (train)5.3s + (eval)12.5s: train loss = 0.24871 = (mf)0.24789 + (embed)0.00083, recall = 0.15972, ndcg = 0.11277, precision = 0.02390, map = 0.09032
Save model to path /kaggle/working/tests/resources/deeprec/lightgcn/model/lightgcn_model/epoch_2
Epoch 2 (train)4.7s + (eval)12.4s: train loss = 0.04636 = (mf)0.04443 + (embed)0.00192, recall = 0.20818, ndcg = 0.15042, precision = 0.03048, map = 0.12296
Save model to path /kaggle/working/tests/resources/deeprec/lightgcn/model/lightgcn_model/epoch_3
Epoch 3 (train)4.6s + (eval)12.5s: train loss = 0.02555 = (mf)0.02307 + (embed)0.00248, recall = 0.23630, ndcg = 0.17411, precision = 0.03424, map = 0.14434
Save model to path /kaggle/working/tests/resources/deeprec/lightgcn/model/lightgcn_model/epoch_4
Epoch 4 (train)4.6s + (eval)12.3s: train loss = 0.01813 = (mf)0.01528 + (embed)0.00284, recall = 0.25495, ndcg = 0.18929, precision = 0.03688

In [19]:
topk_scores = model.recommend_k_items(test, top_k=10, remove_seen=True)

topk_scores.head(20)

,userID,itemID,prediction
0,57513,2565,10.072577
1,57513,54797,8.433524
2,57513,102002,8.211458
3,57513,73968,8.194303
4,57513,120734,6.756838
5,57513,98752,6.482310
6,57513,60544,6.357093
7,57513,23707,6.214141
8,57513,57742,6.206005
9,57513,65261,6.041788


In [20]:
eval_map = map(test, topk_scores, k=10)
eval_ndcg = ndcg_at_k(test, topk_scores, k=10)
eval_precision = precision_at_k(test, topk_scores, k=10)
eval_recall = recall_at_k(test, topk_scores, k=10)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.224181
NDCG:	0.256908
Precision@K:	0.046102
Recall@K:	0.316321


In [21]:
user_file = "/kaggle/working/user_embeddings.csv"
item_file = "/kaggle/working/item_embeddings.csv"

In [22]:
model.infer_embedding(user_file, item_file)

In [24]:
test_1 = test.loc[test['userID'] == 25831]
test_1.head()

,rating,time,itemID,userID
1,5.0,1344211200,55373,25831
2221,4.0,1327881600,56106,25831
2918,4.0,1348272000,3327,25831
6129,4.0,1332892800,110917,25831
8376,5.0,1350518400,53650,25831


In [25]:
topk_scores = model.recommend_k_items(test_1, top_k=10, remove_seen=True)

topk_scores.head(20)

,userID,itemID,prediction
0,25831,100686,18.022717
1,25831,53650,12.479157
2,25831,118874,12.248222
3,25831,36223,12.152377
4,25831,108562,11.908280
5,25831,38300,11.831025
6,25831,4241,11.592193
7,25831,17718,11.189777
8,25831,115313,10.973251
9,25831,21983,10.687944


In [33]:
import shutil

# Specify the directory you want to zip
directory_path = '/kaggle/working/tests/resources/deeprec/lightgcn/model/lightgcn_model/epoch_26'

# Specify the output zip file path (without the .zip extension)
zip_file_path = 'epoch26'

# Create the zip file
shutil.make_archive(zip_file_path, 'zip', directory_path)

'/kaggle/working/epoch26.zip'

In [26]:
books = pd.read_csv('/kaggle/input/books-fil/books_filtered.csv')
books.head(2)

,Title,description,authors,image,previewLink,publisher,publishedDate,categories,id,combined_text_features,embedding
0,Hunting The Hard Way,"Thrilling stories about hunting wildcat, buffa...",['Howard Hill'],http://books.google.com/books/content?id=0lbZC...,http://books.google.nl/books?id=0lbZCRYJXYIC&p...,Derrydale Press,2000-04-26,['Sports & Recreation'],18,thrilling story about hunt wildcat buffalo mou...,[-0.0234614 0.26560947 -0.1578814 -0.157919...
1,Night World: Daughters Of Darkness,"""There’s something strange about the new girls...",['L.J. Smith'],http://books.google.com/books/content?id=c9icD...,http://books.google.nl/books?id=c9icDQAAQBAJ&p...,Simon and Schuster,2016-12-06,['Juvenile Fiction'],55,there s something strange about the new girl i...,[ 0.08108633 0.2756923 -0.21181437 -0.068135...


In [30]:
def get_liked_books_ratings(liked_books, top_n=1):
    # Combine train and test data
    combined_df = pd.concat([train, test], ignore_index=True)

    # Filter combined data by the liked books
    liked_books_ratings = combined_df[combined_df['itemID'].isin(liked_books)]

    # Find users who like multiple books from the liked_books list
    user_liked_books_count = liked_books_ratings.groupby('userID')['itemID'].count().reset_index()
    user_liked_books_count.columns = ['userID', 'liked_books_count']

    # Filter users who liked at least 2 books from the liked_books list
    common_users = user_liked_books_count[user_liked_books_count['liked_books_count'] > 1]['userID']
    
    if not common_users.empty:
        # Filter ratings by common users
        common_users_ratings = liked_books_ratings[liked_books_ratings['userID'].isin(common_users)]
    else:
        # If no common users, get top users for each liked book
        top_users_ratings = pd.DataFrame()
        
        for book_id in liked_books:
            book_ratings = liked_books_ratings[liked_books_ratings['itemID'] == book_id]
            top_users = book_ratings.nlargest(top_n, 'rating')  # Get top users based on rating
            top_users_ratings = pd.concat([top_users_ratings, top_users], ignore_index=True)
        
        common_users_ratings = top_users_ratings

    return common_users_ratings


In [40]:
import pandas as pd

def get_predictions_for_user(id=0, k=10 ,is_new=False, liked_books_ids=[]):
    
    if is_new:
        df = get_liked_books_ratings(liked_books_ids)
    else:
    # Filter the test DataFrame to get the data for the specified user
        df = test.loc[test['userID'] == id]
    
    # Get the top k recommendations for the user, removing items they've already seen
    topk_scores = model.recommend_k_items(df, k, remove_seen=True)
    
    # Ensure topk_scores is ranked by the 'prediction' column
    topk_scores = topk_scores.sort_values(by='prediction', ascending=False)
    
    rec_ids = topk_scores['itemID']
    
#     # Append title IDs of the recommended books
#     for book_id in topk_scores['itemID']:
#         rec_ids.append(test.loc[test['itemID'] == book_id]['itemID'].values[0])
    
    # Retrieve the book titles, authors, and categories from the books DataFrame using the title_id
    recommended_books = books.loc[books['id'].isin(rec_ids), ['Title', 'authors', 'categories']]
    
    return recommended_books


In [33]:
liked_books_ids = [18, 55]  
df = get_liked_books_ratings(liked_books_ids)
df.head()

,rating,time,itemID,userID
0,5.0,1355011200,18,16846
1,5.0,1022976000,55,34971


In [41]:
df = get_predictions_for_user(is_new=True,liked_books_ids = [18, 55] )
df.head(10)

,Title,authors,categories
39,Small-Circle Jujitsu,['Wally Jay'],['Sports & Recreation']
87,The Clan of the Cave Bear,['Jean M. Auel'],['American fiction']
537,"Deep Wizardry: Young Wizard Series, Book 2",['Diane Duane'],['Juvenile Fiction']
751,The Princess Bride,"['Cary Elwes', 'Joe Layden']",['Biography & Autobiography']
1143,Stealing the Network: How to Own a Continent,"['Ryan Russell', 'Joe Grand', 'Tom Craig']",['Computers']
2197,Clan of the Cave Bear (Earth's Children),['Jean M. Auel'],['Fiction']
2300,Inside Photoshop 7 (Inside (New Riders)),['Gary David Bouton'],['Computers']
2495,Chinese Fast Wrestling for Fighting: The Art o...,"['Shou-Yu Liang', 'Tai D. Ngo']",['Art']
4427,Darkfever (Fever Series),['Karen Marie Moning'],['Fiction']
4695,Harry Potter and the Chamber of Secrets,['J. K. Rowling'],['Juvenile Fiction']


In [43]:
df = get_predictions_for_user(0)
df.head(10)

,Title,authors,categories
1989,Jane Eyre (New Windmill),['Charlotte Brontë'],['Charity-schools']
2046,Pride & Prejudice (New Windmill),['Ibi Zoboi'],['Young Adult Fiction']
2232,Pride and Prejudice,['Jane Austen'],['Fiction']
3179,Jane Eyre: Complete and Unabridged (Puffin Cla...,['Charlote Brontë'],['Fiction']
3468,Pride & Prejudice (Classic Library),['Ibi Zoboi'],['Young Adult Fiction']
4967,Jane Eyre (Signet classics),['Charlotte Brontë'],['Fiction']
5402,Frankenstein (Running Press classics),['Mary Wollstonecraft Shelley'],['Frankenstein (Fictitious character)']
9139,Pride & Prejudice (Penguin Classics),['Ibi Zoboi'],['Young Adult Fiction']
9766,Jane Eyre (Large Print),"['Charlotte Brontë', 'Marc Cactus']",['Fiction']
11967,Jane Eyre (Penguin Classics),['Charlotte Bronte'],['Fiction']


In [44]:
print('hello')

hello


In [45]:
combined_df = pd.concat([train, test], ignore_index=True)
combined_df.head()

,rating,time,itemID,userID
0,5.0,1355702400,92408,41587
1,5.0,1356220800,32520,24303
2,4.0,1330560000,112766,56189
3,4.0,1006128000,97297,3294
4,5.0,1290816000,20794,50340


# With full data

In [51]:
data = ImplicitCF(
    train=combined_df,test=test, seed=0,
    col_user='userID',
    col_item='itemID',
    col_rating='rating'
)

In [52]:
model = LightGCN(hparams, data, seed=0)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [54]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

KeyboardInterrupt: 

In [57]:
item_embeddings = pd.read_csv('/kaggle/working/item_embeddings.csv', header=None)
item_embeddings.head()

,0
0,92408\t-0.072949335 0.045973893 -0.22545972 0....
1,32520\t0.36792058 -0.4730116 -0.24796146 0.065...
2,112766\t0.93160146 1.5612078 -0.63366807 -0.22...
3,97297\t-0.54975337 -0.78646016 -0.7324774 -0.3...
4,20794\t-0.026288548 0.83312374 -0.32008472 0.2...


In [77]:
import pandas as pd

# Define the path to the input file
input_file = '/kaggle/working/item_embeddings.csv'

# Initialize lists to store the ids and embeddings
ids = []
embeddings = []

# Read the file line by line
with open(input_file, 'r') as file:
    for line in file:
        # Split the line into id and embedding parts
        parts = line.strip().split('\t')
        id = parts[0]
        embedding_str = parts[1]
        
#         print('id :' + id,'emb: '+ embedding_str)
        # Convert the embedding string to a list of floats
        embedding = embedding_str.split(' ')
        
        # Append the id and embedding to the lists
        ids.append(id)
        embeddings.append(embedding)

# Create a DataFrame from the lists
df = pd.DataFrame({'id': ids, 'str_embedding': embeddings})

# Save the structured DataFrame to a new CSV file
df.to_csv('structured_item_embeddings.csv', index=False)

# Display the DataFrame
print(df.head())


       id                                      str_embedding
0   92408  [-0.072949335, 0.045973893, -0.22545972, 0.852...
1   32520  [0.36792058, -0.4730116, -0.24796146, 0.065620...
2  112766  [0.93160146, 1.5612078, -0.63366807, -0.227508...
3   97297  [-0.54975337, -0.78646016, -0.7324774, -0.3064...
4   20794  [-0.026288548, 0.83312374, -0.32008472, 0.2779...


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12584 entries, 0 to 12583
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             12584 non-null  object
 1   str_embedding  12584 non-null  object
dtypes: object(2)
memory usage: 196.8+ KB


In [79]:
type(float(df['str_embedding'].loc[0][0]))

float

In [87]:
# Convert 'id' column to integers
df['id'] = pd.to_numeric(df['id'], errors='coerce').astype('Int64')

In [85]:
embeddings = []
for str_embedding in df['str_embedding']:
    embeddings.append(np.asarray(str_embedding, dtype=float))

In [88]:
df['embedding'] = embeddings

In [89]:
df.head()

,id,str_embedding,embedding
0,92408,"[-0.072949335, 0.045973893, -0.22545972, 0.852...","[-0.072949335, 0.045973893, -0.22545972, 0.852..."
1,32520,"[0.36792058, -0.4730116, -0.24796146, 0.065620...","[0.36792058, -0.4730116, -0.24796146, 0.065620..."
2,112766,"[0.93160146, 1.5612078, -0.63366807, -0.227508...","[0.93160146, 1.5612078, -0.63366807, -0.227508..."
3,97297,"[-0.54975337, -0.78646016, -0.7324774, -0.3064...","[-0.54975337, -0.78646016, -0.7324774, -0.3064..."
4,20794,"[-0.026288548, 0.83312374, -0.32008472, 0.2779...","[-0.026288548, 0.83312374, -0.32008472, 0.2779..."


In [91]:
type(float(df['embedding'].loc[0][0]))

float

In [93]:
df.drop(columns=['str_embedding'], inplace=True)

In [94]:
df.head()

,id,embedding
0,92408,"[-0.072949335, 0.045973893, -0.22545972, 0.852..."
1,32520,"[0.36792058, -0.4730116, -0.24796146, 0.065620..."
2,112766,"[0.93160146, 1.5612078, -0.63366807, -0.227508..."
3,97297,"[-0.54975337, -0.78646016, -0.7324774, -0.3064..."
4,20794,"[-0.026288548, 0.83312374, -0.32008472, 0.2779..."


In [95]:
# Save the updated DataFrame to a new CSV file if needed
df.to_csv('updated_item_embeddings.csv', index=False)

In [96]:
len(df['embedding'].loc[0])

64